In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://keras.io/examples/nlp/text_classification_with_transformer/

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from sklearn.model_selection import train_test_split

In [3]:
pwd_train_40 = '/content/drive/MyDrive/Colab Notebooks/NLP proj/data_thresh40/train_data.csv'
pwd_val_40 = '/content/drive/MyDrive/Colab Notebooks/NLP proj/data_thresh40/val_data.csv'
pwd_test_40 = '/content/drive/MyDrive/Colab Notebooks/NLP proj/data_thresh40/test_data.csv'

train_df = pd.read_csv(pwd_train_40)
val_df = pd.read_csv(pwd_val_40)
test_df = pd.read_csv(pwd_test_40)

In [4]:
X_train = train_df['clean_content']
X_val = val_df['clean_content']
X_test = test_df['clean_content']

In [5]:
#Get cleaned input sequence
t=[]
for i in X_train:
  t.append(i.strip('!"#$%&()*+,-.–—/:;<=>?@[\\]^_`{|}~\t\r\n'))       
v=[]
for i in X_val:
  v.append(i.strip('!"#$%&()*+,-.–—/:;<=>?@[\\]^_`{|}~\t\r\n'))
e=[]
for i in X_test:
  e.append(i.strip('!"#$%&()*+,-.–—/:;<=>?@[\\]^_`{|}~\t\r\n'))

x_train=[]
for i in range(len(t)):
  x_train.append(' '.join(t[i].split('\n')))
x_val=[]
for i in range(len(v)):
  x_val.append(' '.join(v[i].split('\n')))
x_test=[]
for i in range(len(e)):
  x_test.append(' '.join(e[i].split('\n')))
print(len(x_train),len(x_val),len(x_test))

x_train = [i.split(' ') for i in x_train]
x_test = [i.split(' ') for i in x_test]
xv=[]
for i in x_val:
  xv.append(i.split(' '))
x_val=xv

457 127 59


In [6]:
#check input
print(x_train[4])
print(x_val[5])
print(x_test[2])

['sometim', 'someth', 'like', 'second', 'wash', 'base', 'street', 'th', 'father', 'two', 'assistants', 'ar', 'given', 'permiss', 'go', 'on', 'woman', 'ask', '“why', 'did', 'come', 'first', 'place', 'to', 'citadel', 'dampness”', '', 'som', 'day', 'wors', 'others', 'even', "can't", 'believ', 'them', 'but', 'never', 'concern', 'mine', 'reason', 'patient', '', 's', 'scroll', 'never', 'blast', 'us', 'into', 'marmor', 'mean', 'fist', 'it', 'kudo', 'princ', 'journey', 'here', 'to', 'negoti', 'releas', 'believ', 'it', '', "you'r", 'right', 'ballad', 'retreating', 'back', 'atmosphere', 'they', "won't", 'come', 'round', 'again', 'mak', 'peac']
['mani', 'time', 'low', 'foot', 'stagger', 'solder', 'mouth', 'tell', 'tri', 'stir', 'aw', 'rivet', 'tri', 'lift', 'hasp', 'steel', '', 'strok', 'cool', 'forehead', 'hot', 'often', 'lift', 'care', 'listless', 'hair', 'handl', 'adamantin', 'fingers', 'nev', 'thimbl', 'shall', 'wear', 'buzz', 'dull', 'fli', 'chamber', 'window', 'brave', 'shine', 'sun', 'frec

In [7]:
#(Optional) Splitting input sequence
max_len = 150
author_train=[]
poem_train=[]
for k in range(len(x_train)):
  x = x_train[k]
  if len(x) > max_len:  #Can choose different length
    chunks, chunk_size = len(x), max_len
    a=[x[i:i+int(chunk_size)] for i in range(0, int(chunks), int(chunk_size)) ]
    for j in a:
      poem_train.append(j)
      author_train.append(train_df['author'][k])
  else:
    poem_train.append(x)
    author_train.append(train_df['author'][k])

author_val=[]
poem_val=[]
for k in range(len(x_val)):
  x = x_val[k]
  if len(x) > max_len:
    chunks, chunk_size = len(x), max_len
    a=[x[i:i+int(chunk_size)] for i in range(0, int(chunks), int(chunk_size)) ]
    for j in a:
      poem_val.append(j)
      author_val.append(val_df['author'][k])
  else:
    poem_val.append(x)
    author_val.append(val_df['author'][k])

author_test=[]
poem_test=[]
for k in range(len(x_test)):
  x = x_test[k]
  if len(x) > max_len:
    chunks, chunk_size = len(x), max_len
    a=[x[i:i+int(chunk_size)] for i in range(0, int(chunks), int(chunk_size)) ]
    for j in a:
      poem_test.append(j)
      author_test.append(test_df['author'][k])
  else:
    poem_test.append(x)
    author_test.append(test_df['author'][k])

print(len(author_train)==len(poem_train))
print(len(author_val)==len(poem_val))    
print(len(author_test)==len(poem_test))

True
True
True


In [ ]:
#Check distribution
from collections import Counter
Counter(author_train)

Counter({'Alfred, Lord Tennyson': 151,
         'Emily Dickinson': 40,
         'John Ashbery': 57,
         'John Donne': 56,
         'Kay Ryan': 29,
         'Percy sshe Shelley': 110,
         'Rae Armantrout': 46,
         'Walt Whitman': 128,
         'William Butler Yeats': 46,
         'William Shakespeare': 109,
         'William Wordsworth': 106,
         'Yusef Komunyakaa': 37})

In [8]:
# Tokenize the dataset
MAX_NB_WORDS = 50000
oov_token = "<UNK>"
padding_type = "post"
trunction_type='post'
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, oov_token=oov_token,filters='!"#$%&()*+,-–./:;<=>?@[\\]^_`{|}~\t\n\r')
tokenizer.fit_on_texts(poem_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15310 unique tokens.


In [9]:
X_train_sequences = tokenizer.texts_to_sequences(poem_train)
X_train_padded = pad_sequences(X_train_sequences, padding=padding_type, 
                       truncating=trunction_type)
X_val_sequences = tokenizer.texts_to_sequences(poem_val)
X_val_padded = pad_sequences(X_val_sequences, padding=padding_type,
                       truncating=trunction_type)

X_test_sequences = tokenizer.texts_to_sequences(poem_test)
X_test_padded = pad_sequences(X_test_sequences, padding=padding_type, 
                       truncating=trunction_type)

In [10]:
X_train_padded.shape

(915, 150)

In [11]:
#Encode label to numbers  (Use 'SparseCategoricalCrossentropy' if using numbers)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(author_train)
y_train=le.transform(author_train)
y_val=le.transform(author_val)
y_test=le.transform(author_test)

In [12]:
y_train.shape

(915,)

In [ ]:
from keras.utils import to_categorical
to_categorical(y_train)

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
to_categorical(y_train).shape

(915, 12)

In [ ]:
#didn't run 
#(Optional)One hot encoding   (Use 'categorical_crossentropy' if using one hot)
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)


In [13]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output


In [14]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [15]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


w/o lr decay, max_len = 150,dim = 32, num_heads = 4

In [42]:
embed_dim = 128  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
nclasses = 12
max_length = 150

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, MAX_NB_WORDS, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [43]:
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=1e-2,
#    decay_steps=10000,
#    decay_rate=0.9)
#opt = keras.optimizers.Adam(learning_rate=lr_schedule)
#model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])

model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])


In [44]:
#run 1
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=30, validation_data=(X_val_padded, y_val)
)

Epoch 1/30
29/29 [==============================] - 7s 226ms/step - loss: 2.4058 - accuracy: 0.1650 - val_loss: 2.5276 - val_accuracy: 0.0479
Epoch 2/30
29/29 [==============================] - 6s 215ms/step - loss: 2.0492 - accuracy: 0.2929 - val_loss: 2.2573 - val_accuracy: 0.1809
Epoch 3/30
29/29 [==============================] - 6s 212ms/step - loss: 1.6036 - accuracy: 0.4437 - val_loss: 2.0604 - val_accuracy: 0.3138
Epoch 4/30
29/29 [==============================] - 6s 212ms/step - loss: 1.1108 - accuracy: 0.6164 - val_loss: 1.8304 - val_accuracy: 0.3617
Epoch 5/30
29/29 [==============================] - 6s 211ms/step - loss: 0.8108 - accuracy: 0.7268 - val_loss: 1.6793 - val_accuracy: 0.5000
Epoch 6/30
29/29 [==============================] - 6s 216ms/step - loss: 0.4775 - accuracy: 0.8426 - val_loss: 1.7462 - val_accuracy: 0.5319
Epoch 7/30
29/29 [==============================] - 6s 213ms/step - loss: 0.3128 - accuracy: 0.9027 - val_loss: 1.6328 - val_accuracy: 0.6064
Epoch 

In [26]:
#run 2
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=30, validation_data=(X_val_padded, y_val)
)

Epoch 1/30
29/29 [==============================] - 3s 100ms/step - loss: 2.3990 - accuracy: 0.1344 - val_loss: 2.3836 - val_accuracy: 0.1170
Epoch 2/30
29/29 [==============================] - 3s 103ms/step - loss: 2.1917 - accuracy: 0.2317 - val_loss: 2.3803 - val_accuracy: 0.1383
Epoch 3/30
29/29 [==============================] - 4s 155ms/step - loss: 1.8384 - accuracy: 0.4022 - val_loss: 2.0140 - val_accuracy: 0.3883
Epoch 4/30
29/29 [==============================] - 5s 155ms/step - loss: 1.3707 - accuracy: 0.5836 - val_loss: 1.8432 - val_accuracy: 0.3723
Epoch 5/30
29/29 [==============================] - 4s 145ms/step - loss: 1.0113 - accuracy: 0.6721 - val_loss: 1.6584 - val_accuracy: 0.4043
Epoch 6/30
29/29 [==============================] - 3s 89ms/step - loss: 0.7694 - accuracy: 0.7814 - val_loss: 1.4580 - val_accuracy: 0.5053
Epoch 7/30
29/29 [==============================] - 3s 89ms/step - loss: 0.5261 - accuracy: 0.8623 - val_loss: 1.3498 - val_accuracy: 0.5426
Epoch 8/

In [38]:
model.evaluate(X_test_padded, y_test)

4/4 [==============================] - 0s 30ms/step - loss: 2.6893 - accuracy: 0.2600


[2.6892850399017334, 0.25999999046325684]

In [39]:
authors = list(set(author_train))

In [40]:
authors = sorted(authors)

In [41]:
for i in range(12):
  print (authors[i])
  ind = np.where(y_test == i)
  X = X_test_padded[ind]
  y = y_test[ind]
  model.evaluate(X, y)


Alfred, Lord Tennyson
1/1 [==============================] - 0s 1ms/step - loss: 1.8214 - accuracy: 0.6000
Emily Dickinson
1/1 [==============================] - 0s 3ms/step - loss: 2.3337 - accuracy: 0.2000
John Ashbery
1/1 [==============================] - 0s 1ms/step - loss: 2.4439 - accuracy: 0.0000e+00
John Donne
1/1 [==============================] - 0s 2ms/step - loss: 1.3935 - accuracy: 0.6000
Kay Ryan
1/1 [==============================] - 0s 2ms/step - loss: 2.5769 - accuracy: 0.0000e+00
Percy sshe Shelley
1/1 [==============================] - 0s 1ms/step - loss: 4.0196 - accuracy: 0.2258
Rae Armantrout
1/1 [==============================] - 0s 1ms/step - loss: 0.9568 - accuracy: 0.5000
Walt Whitman
1/1 [==============================] - 0s 1ms/step - loss: 2.8129 - accuracy: 0.0000e+00
William Butler Yeats
1/1 [==============================] - 0s 2ms/step - loss: 2.0221 - accuracy: 0.4000
William Shakespeare
1/1 [==============================] - 0s 1ms/step - loss: 1.803

In [ ]:
#run 3
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=40, validation_data=(X_val_padded, y_val)
)

Epoch 1/40
29/29 [==============================] - 3s 96ms/step - loss: 2.4758 - accuracy: 0.1290 - val_loss: 2.4026 - val_accuracy: 0.1436
Epoch 2/40
29/29 [==============================] - 2s 83ms/step - loss: 2.2199 - accuracy: 0.2317 - val_loss: 2.3484 - val_accuracy: 0.1330
Epoch 3/40
29/29 [==============================] - 2s 83ms/step - loss: 1.8766 - accuracy: 0.4142 - val_loss: 2.2240 - val_accuracy: 0.1862
Epoch 4/40
29/29 [==============================] - 2s 85ms/step - loss: 1.3804 - accuracy: 0.5683 - val_loss: 1.9379 - val_accuracy: 0.3085
Epoch 5/40
29/29 [==============================] - 2s 84ms/step - loss: 1.0109 - accuracy: 0.6940 - val_loss: 1.7065 - val_accuracy: 0.4255
Epoch 6/40
29/29 [==============================] - 2s 84ms/step - loss: 0.7635 - accuracy: 0.7694 - val_loss: 1.7372 - val_accuracy: 0.3936
Epoch 7/40
29/29 [==============================] - 2s 84ms/step - loss: 0.5779 - accuracy: 0.8393 - val_loss: 1.6172 - val_accuracy: 0.4628
Epoch 8/40
29

w/o lr decay, max_len = 150,dim = 64, num_heads = 4

In [ ]:
embed_dim = 128  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
nclasses = 12
max_length = 150

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, MAX_NB_WORDS, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=1e-2,
#    decay_steps=10000,
#    decay_rate=0.9)
#opt = keras.optimizers.Adam(learning_rate=lr_schedule)
#model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])

model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=40, validation_data=(X_val_padded, y_val)
)

Epoch 1/40
29/29 [==============================] - 4s 133ms/step - loss: 2.3357 - accuracy: 0.1475 - val_loss: 2.3416 - val_accuracy: 0.2021
Epoch 2/40
29/29 [==============================] - 4s 121ms/step - loss: 2.0916 - accuracy: 0.2863 - val_loss: 2.1950 - val_accuracy: 0.3245
Epoch 3/40
29/29 [==============================] - 4s 122ms/step - loss: 1.6030 - accuracy: 0.5344 - val_loss: 1.8978 - val_accuracy: 0.3670
Epoch 4/40
29/29 [==============================] - 4s 122ms/step - loss: 1.0749 - accuracy: 0.6852 - val_loss: 1.5503 - val_accuracy: 0.4681
Epoch 5/40
29/29 [==============================] - 4s 122ms/step - loss: 0.6485 - accuracy: 0.8077 - val_loss: 1.7051 - val_accuracy: 0.4521
Epoch 6/40
29/29 [==============================] - 4s 121ms/step - loss: 0.4721 - accuracy: 0.8634 - val_loss: 1.3954 - val_accuracy: 0.5957
Epoch 7/40
29/29 [==============================] - 4s 124ms/step - loss: 0.3403 - accuracy: 0.9104 - val_loss: 1.2219 - val_accuracy: 0.6170
Epoch 

w/o lr decay, max_len = 150,dim = 64, num_heads = 2

In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=40, validation_data=(X_val_padded, y_val)
)

Epoch 1/40
29/29 [==============================] - 3s 116ms/step - loss: 2.3087 - accuracy: 0.1989 - val_loss: 2.3134 - val_accuracy: 0.1968
Epoch 2/40
29/29 [==============================] - 3s 106ms/step - loss: 1.9141 - accuracy: 0.3639 - val_loss: 2.1415 - val_accuracy: 0.3457
Epoch 3/40
29/29 [==============================] - 3s 106ms/step - loss: 1.4234 - accuracy: 0.5443 - val_loss: 1.9115 - val_accuracy: 0.3245
Epoch 4/40
29/29 [==============================] - 3s 105ms/step - loss: 0.9921 - accuracy: 0.6995 - val_loss: 1.6582 - val_accuracy: 0.4574
Epoch 5/40
29/29 [==============================] - 3s 105ms/step - loss: 0.6076 - accuracy: 0.8295 - val_loss: 1.5283 - val_accuracy: 0.5000
Epoch 6/40
29/29 [==============================] - 3s 107ms/step - loss: 0.4331 - accuracy: 0.8896 - val_loss: 1.4212 - val_accuracy: 0.5426
Epoch 7/40
29/29 [==============================] - 3s 106ms/step - loss: 0.3132 - accuracy: 0.9137 - val_loss: 1.2843 - val_accuracy: 0.5904
Epoch 

w/o lr decay, max_len = 150,dim = 128, num_heads = 4

In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
29/29 [==============================] - 7s 232ms/step - loss: 2.3103 - accuracy: 0.1749 - val_loss: 2.3649 - val_accuracy: 0.1596
Epoch 2/25
29/29 [==============================] - 6s 221ms/step - loss: 1.8363 - accuracy: 0.3530 - val_loss: 2.0743 - val_accuracy: 0.2819
Epoch 3/25
29/29 [==============================] - 6s 222ms/step - loss: 1.3337 - accuracy: 0.5607 - val_loss: 1.7562 - val_accuracy: 0.3883
Epoch 4/25
29/29 [==============================] - 6s 220ms/step - loss: 0.8714 - accuracy: 0.7158 - val_loss: 1.6192 - val_accuracy: 0.4362
Epoch 5/25
29/29 [==============================] - 6s 222ms/step - loss: 0.5323 - accuracy: 0.8426 - val_loss: 1.7111 - val_accuracy: 0.4362
Epoch 6/25
29/29 [==============================] - 6s 221ms/step - loss: 0.3649 - accuracy: 0.8863 - val_loss: 1.4080 - val_accuracy: 0.5266
Epoch 7/25
29/29 [==============================] - 6s 222ms/step - loss: 0.2117 - accuracy: 0.9388 - val_loss: 1.7084 - val_accuracy: 0.4894
Epoch 

with lr decay, max_len = 150,dim = 128, num_heads = 4

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=1000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
29/29 [==============================] - 7s 232ms/step - loss: 2.4251 - accuracy: 0.1377 - val_loss: 2.3907 - val_accuracy: 0.1809
Epoch 2/25
29/29 [==============================] - 6s 220ms/step - loss: 2.1420 - accuracy: 0.2197 - val_loss: 2.3137 - val_accuracy: 0.1383
Epoch 3/25
29/29 [==============================] - 6s 221ms/step - loss: 1.9879 - accuracy: 0.2754 - val_loss: 2.1588 - val_accuracy: 0.2872
Epoch 4/25
29/29 [==============================] - 6s 222ms/step - loss: 1.7653 - accuracy: 0.3607 - val_loss: 2.1651 - val_accuracy: 0.2447
Epoch 5/25
29/29 [==============================] - 6s 221ms/step - loss: 1.6681 - accuracy: 0.3934 - val_loss: 2.0865 - val_accuracy: 0.2447
Epoch 6/25
29/29 [==============================] - 6s 222ms/step - loss: 1.5096 - accuracy: 0.4383 - val_loss: 1.9808 - val_accuracy: 0.2766
Epoch 7/25
29/29 [==============================] - 6s 218ms/step - loss: 1.4133 - accuracy: 0.4754 - val_loss: 2.1417 - val_accuracy: 0.2553
Epoch 

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
29/29 [==============================] - 7s 242ms/step - loss: 2.3882 - accuracy: 0.1257 - val_loss: 2.3977 - val_accuracy: 0.1117
Epoch 2/25
29/29 [==============================] - 7s 239ms/step - loss: 2.0083 - accuracy: 0.3246 - val_loss: 2.1628 - val_accuracy: 0.2234
Epoch 3/25
29/29 [==============================] - 11s 390ms/step - loss: 1.4364 - accuracy: 0.5093 - val_loss: 1.7435 - val_accuracy: 0.4149
Epoch 4/25
29/29 [==============================] - 6s 223ms/step - loss: 0.8508 - accuracy: 0.7432 - val_loss: 1.4645 - val_accuracy: 0.5160
Epoch 5/25
29/29 [==============================] - 6s 220ms/step - loss: 0.4635 - accuracy: 0.8623 - val_loss: 1.6009 - val_accuracy: 0.4894
Epoch 6/25
29/29 [==============================] - 6s 220ms/step - loss: 0.2878 - accuracy: 0.9148 - val_loss: 1.3396 - val_accuracy: 0.5691
Epoch 7/25
29/29 [==============================] - 6s 221ms/step - loss: 0.1530 - accuracy: 0.9585 - val_loss: 1.4219 - val_accuracy: 0.6064
Epoch

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
29/29 [==============================] - 7s 235ms/step - loss: 2.2867 - accuracy: 0.1672 - val_loss: 2.3329 - val_accuracy: 0.1968
Epoch 2/25
29/29 [==============================] - 6s 223ms/step - loss: 1.8709 - accuracy: 0.3322 - val_loss: 2.1371 - val_accuracy: 0.2234
Epoch 3/25
29/29 [==============================] - 6s 220ms/step - loss: 1.3090 - accuracy: 0.5355 - val_loss: 1.8750 - val_accuracy: 0.3457
Epoch 4/25
29/29 [==============================] - 6s 223ms/step - loss: 0.9384 - accuracy: 0.6743 - val_loss: 2.0346 - val_accuracy: 0.3245
Epoch 5/25
29/29 [==============================] - 6s 222ms/step - loss: 0.7904 - accuracy: 0.7454 - val_loss: 1.6706 - val_accuracy: 0.4309
Epoch 6/25
29/29 [==============================] - 6s 223ms/step - loss: 0.5042 - accuracy: 0.8437 - val_loss: 1.5174 - val_accuracy: 0.5213
Epoch 7/25
29/29 [==============================] - 6s 223ms/step - loss: 0.3101 - accuracy: 0.9082 - val_loss: 1.3280 - val_accuracy: 0.5691
Epoch 

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
29/29 [==============================] - 7s 234ms/step - loss: 2.3233 - accuracy: 0.1967 - val_loss: 2.4101 - val_accuracy: 0.1011
Epoch 2/25
29/29 [==============================] - 6s 221ms/step - loss: 1.8642 - accuracy: 0.3268 - val_loss: 2.2243 - val_accuracy: 0.2128
Epoch 3/25
29/29 [==============================] - 6s 223ms/step - loss: 1.4654 - accuracy: 0.4798 - val_loss: 2.0250 - val_accuracy: 0.3245
Epoch 4/25
29/29 [==============================] - 6s 223ms/step - loss: 1.0587 - accuracy: 0.6426 - val_loss: 1.7950 - val_accuracy: 0.4415
Epoch 5/25
29/29 [==============================] - 6s 223ms/step - loss: 0.6896 - accuracy: 0.7661 - val_loss: 1.8636 - val_accuracy: 0.4787
Epoch 6/25
29/29 [==============================] - 6s 222ms/step - loss: 0.4459 - accuracy: 0.8601 - val_loss: 1.5842 - val_accuracy: 0.4894
Epoch 7/25
29/29 [==============================] - 6s 223ms/step - loss: 0.2932 - accuracy: 0.9082 - val_loss: 1.3430 - val_accuracy: 0.5851
Epoch 

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=150,
    decay_rate=1)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
29/29 [==============================] - 7s 241ms/step - loss: 0.0146 - accuracy: 0.9978 - val_loss: 2.6886 - val_accuracy: 0.5957
Epoch 2/25
29/29 [==============================] - 7s 224ms/step - loss: 0.0210 - accuracy: 0.9956 - val_loss: 2.6163 - val_accuracy: 0.6011
Epoch 3/25
29/29 [==============================] - 7s 225ms/step - loss: 0.0117 - accuracy: 0.9956 - val_loss: 2.4874 - val_accuracy: 0.6489
Epoch 4/25
29/29 [==============================] - 6s 223ms/step - loss: 0.0153 - accuracy: 0.9956 - val_loss: 2.5111 - val_accuracy: 0.6702
Epoch 5/25
29/29 [==============================] - 6s 223ms/step - loss: 0.0109 - accuracy: 0.9978 - val_loss: 2.6654 - val_accuracy: 0.6489
Epoch 6/25
29/29 [==============================] - 6s 222ms/step - loss: 0.0114 - accuracy: 0.9967 - val_loss: 2.6052 - val_accuracy: 0.6436
Epoch 7/25
29/29 [==============================] - 7s 227ms/step - loss: 0.0140 - accuracy: 0.9956 - val_loss: 3.1937 - val_accuracy: 0.5851
Epoch 

with lr decay, max_len = 200, dim = 128, num_heads = 4

In [ ]:
embed_dim = 128  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
nclasses = 12
max_length = 200

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, MAX_NB_WORDS, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=200,
    decay_rate=1)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=40, validation_data=(X_val_padded, y_val)
)

Epoch 1/40
25/25 [==============================] - 8s 303ms/step - loss: 2.3809 - accuracy: 0.1688 - val_loss: 2.3878 - val_accuracy: 0.2229
Epoch 2/40
25/25 [==============================] - 7s 286ms/step - loss: 2.1127 - accuracy: 0.2668 - val_loss: 2.3590 - val_accuracy: 0.1446
Epoch 3/40
25/25 [==============================] - 7s 286ms/step - loss: 1.7051 - accuracy: 0.4253 - val_loss: 2.1779 - val_accuracy: 0.1988
Epoch 4/40
25/25 [==============================] - 7s 286ms/step - loss: 1.3108 - accuracy: 0.5438 - val_loss: 2.2924 - val_accuracy: 0.2651
Epoch 5/40
25/25 [==============================] - 7s 290ms/step - loss: 0.9531 - accuracy: 0.6843 - val_loss: 1.8892 - val_accuracy: 0.3434
Epoch 6/40
25/25 [==============================] - 7s 290ms/step - loss: 0.6167 - accuracy: 0.7874 - val_loss: 1.6403 - val_accuracy: 0.4518
Epoch 7/40
25/25 [==============================] - 7s 285ms/step - loss: 0.3411 - accuracy: 0.8982 - val_loss: 1.5332 - val_accuracy: 0.4940
Epoch 

w/o lr decay, max_len = 200, dim = 128, num_heads = 4

In [ ]:
embed_dim = 128  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
nclasses = 12
max_length = 200

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, MAX_NB_WORDS, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# run 1
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
25/25 [==============================] - 7s 299ms/step - loss: 2.3807 - accuracy: 0.1430 - val_loss: 2.3377 - val_accuracy: 0.1506
Epoch 2/25
25/25 [==============================] - 7s 283ms/step - loss: 2.0525 - accuracy: 0.2642 - val_loss: 2.2315 - val_accuracy: 0.2349
Epoch 3/25
25/25 [==============================] - 7s 285ms/step - loss: 1.6810 - accuracy: 0.4394 - val_loss: 2.0335 - val_accuracy: 0.2530
Epoch 4/25
25/25 [==============================] - 7s 284ms/step - loss: 1.3839 - accuracy: 0.5103 - val_loss: 1.9282 - val_accuracy: 0.2771
Epoch 5/25
25/25 [==============================] - 7s 285ms/step - loss: 1.0836 - accuracy: 0.6237 - val_loss: 1.6227 - val_accuracy: 0.4458
Epoch 6/25
25/25 [==============================] - 7s 286ms/step - loss: 0.7157 - accuracy: 0.7668 - val_loss: 1.5508 - val_accuracy: 0.5482
Epoch 7/25
25/25 [==============================] - 7s 284ms/step - loss: 0.4070 - accuracy: 0.8763 - val_loss: 1.2664 - val_accuracy: 0.6506
Epoch 

In [ ]:
# run 2
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
25/25 [==============================] - 8s 300ms/step - loss: 2.3203 - accuracy: 0.1559 - val_loss: 2.3278 - val_accuracy: 0.1566
Epoch 2/25
25/25 [==============================] - 7s 286ms/step - loss: 2.0382 - accuracy: 0.2977 - val_loss: 2.2145 - val_accuracy: 0.3072
Epoch 3/25
25/25 [==============================] - 7s 289ms/step - loss: 1.6704 - accuracy: 0.4304 - val_loss: 2.0630 - val_accuracy: 0.2952
Epoch 4/25
25/25 [==============================] - 7s 290ms/step - loss: 1.3053 - accuracy: 0.5709 - val_loss: 1.7473 - val_accuracy: 0.4578
Epoch 5/25
25/25 [==============================] - 7s 291ms/step - loss: 0.9477 - accuracy: 0.6740 - val_loss: 1.3738 - val_accuracy: 0.5422
Epoch 6/25
25/25 [==============================] - 7s 290ms/step - loss: 0.5424 - accuracy: 0.8325 - val_loss: 1.5209 - val_accuracy: 0.5422
Epoch 7/25
25/25 [==============================] - 7s 291ms/step - loss: 0.3161 - accuracy: 0.9124 - val_loss: 1.2113 - val_accuracy: 0.6205
Epoch 

w/o lr decay, max_len = 200, dim = 128, num_heads = 4, 2 transformer blocksa

In [ ]:
embed_dim = 256  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 256  # Hidden layer size in feed forward network inside transformer
nclasses = 12
max_length = 200

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, MAX_NB_WORDS, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
#x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# with 2 transformer blocks
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
25/25 [==============================] - 13s 519ms/step - loss: 2.4088 - accuracy: 0.1366 - val_loss: 2.3828 - val_accuracy: 0.1687
Epoch 2/25
25/25 [==============================] - 13s 513ms/step - loss: 2.0531 - accuracy: 0.2771 - val_loss: 2.2294 - val_accuracy: 0.1988
Epoch 3/25
25/25 [==============================] - 12s 498ms/step - loss: 1.5011 - accuracy: 0.4755 - val_loss: 2.2174 - val_accuracy: 0.2410
Epoch 4/25
25/25 [==============================] - 12s 499ms/step - loss: 0.9994 - accuracy: 0.6521 - val_loss: 1.8183 - val_accuracy: 0.3735
Epoch 5/25
25/25 [==============================] - 13s 502ms/step - loss: 0.6334 - accuracy: 0.8144 - val_loss: 1.6696 - val_accuracy: 0.4940
Epoch 6/25
25/25 [==============================] - 13s 501ms/step - loss: 0.4296 - accuracy: 0.8724 - val_loss: 1.8936 - val_accuracy: 0.4578
Epoch 7/25
25/25 [==============================] - 12s 500ms/step - loss: 0.2388 - accuracy: 0.9317 - val_loss: 1.7391 - val_accuracy: 0.4880

w/o lr decay, max_len = 200, dim = 256, num_heads = 4

In [ ]:
model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:

history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=25, validation_data=(X_val_padded, y_val)
)

Epoch 1/25
25/25 [==============================] - 16s 632ms/step - loss: 2.4022 - accuracy: 0.1340 - val_loss: 2.3934 - val_accuracy: 0.1566
Epoch 2/25
25/25 [==============================] - 16s 621ms/step - loss: 2.1571 - accuracy: 0.1534 - val_loss: 2.3931 - val_accuracy: 0.1265
Epoch 3/25
25/25 [==============================] - 15s 619ms/step - loss: 1.9505 - accuracy: 0.2629 - val_loss: 2.2379 - val_accuracy: 0.1386
Epoch 4/25
25/25 [==============================] - 16s 620ms/step - loss: 1.6993 - accuracy: 0.3634 - val_loss: 2.2425 - val_accuracy: 0.1988
Epoch 5/25
25/25 [==============================] - 15s 618ms/step - loss: 1.4831 - accuracy: 0.4536 - val_loss: 2.1491 - val_accuracy: 0.2590
Epoch 6/25
25/25 [==============================] - 15s 619ms/step - loss: 1.1758 - accuracy: 0.5528 - val_loss: 1.5069 - val_accuracy: 0.4277
Epoch 7/25
25/25 [==============================] - 15s 619ms/step - loss: 0.9187 - accuracy: 0.6302 - val_loss: 1.6273 - val_accuracy: 0.3916

### with glove embedding
not working lol

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


--2020-11-23 20:01:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-23 20:01:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-23 20:01:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
! ls

drive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip


In [ ]:
import os
import pathlib


In [ ]:
embeddings_index = {}
with open('/content/glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))


Found 400000 word vectors.


In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))

for word, i in word_index.items():

    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
glove_embedding_layer = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

<tf.Tensor 'embedding_25/embedding_lookup_1/Identity_1:0' shape=(None, 20, 300) dtype=float32>

In [ ]:
from keras.layers import Flatten

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
nclasses = 12

inputs = layers.Input(shape=(max_length,))
#print(inputs.shape)
embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
x = embedding_layer(inputs)
print(x.shape)
x = glove_embedding_layer(x)
print(x.shape)
#x = Flatten(x)
#print(x.shape)
#transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
transformer_block = TransformerBlock(100, num_heads, 100)
x = transformer_block(x)
print(x.shape)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


(None, 300, 32)
(None, 300, 32, 100)
(None, 300, 32, 100)


In [ ]:
model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=40, validation_data=(X_val_padded, y_val)
)

Epoch 1/40
